# Preparing Management Accounts
Starting with a YTD TTB you need to add the begining of the year trial balance.    

Note that during year during the first months we will not have the accountants adjustments and at some point we will.  At that point there will be a discontinuity in the accounting record.   

If you are preparing part way through the year then you may take a mid year point as the starting point for the preperation of the accounts.  This happend in 2014-2015 and September 2015 was used as a starting point or the subsequent months of the year.

In the first instance this starting point would be the January of the preceding financial year.


## Get the Jan 15 accounts

In [1]:
import datetime as dt
import math
import pandas as pd
import pickle
import sqlite3

from luca import SageData, TransactionalTrialBalance, p, PeriodReport, ExcelManagementReport,\
    LoadDatabaseError, LoadDatabase, chart_of_accounts_from_db, journal_from_db, TrialBalance

In [2]:
rep = PeriodReport(dt.datetime(2015, 1, 1), "historic_trial_balances.db")

In [3]:
emr = ExcelManagementReport()
emr.create_one_report(rep)
rep.period_ytd.to_series().head()

10        9926.00
1001    362381.70
1100    303456.80
1102         0.00
1103    -30274.58
dtype: object

In [4]:
for attr in ('period_mtd', 'period_ytd', 'prior_mtd', 'prior_ytd'):
    print('Sum for {} = {}'.format(attr, getattr(rep, attr).sum()))

Sum for period_mtd = -75044.01
Sum for period_ytd = 0.00
Sum for prior_mtd = -107345.59
Sum for prior_ytd = 0.00


## Convert YTD for prior year to start of year
We need to close out the year by clearning the P&L and adding to the retained profit for the year.  (Note this doesn't have the accountants changes which we will need for Jan 16).   
rep.period_ytd  --> to year_start.   
Sum > 3000 and add to retained profit 2125
Clear all > 3000

In [5]:
start_year=rep.period_ytd.close_period()
start_year.to_series().head()

P&L = -45685.49
Prev retain = -509969.50, changing to -555654.99
New retained profit = -555654.99
Sum before clearing old balances = -45685.49
Sum after clearing old balances = 0.00


10        9926.00
1001    362381.70
1100    303456.80
1102         0.00
1103    -30274.58
dtype: object

## Add start of year to TTB
In order to create the accounts for the YTD we need to add the TTB for the YTD to the year start.   
This should be a generalised TB add.

In [6]:
sage_2015 = pickle.load(open('sage2015.pickle', 'rb'))

In [7]:
ttb = sage_2015.transactional_trial_balance(dt.date(2015, 2, 1), dt.date(2015, 2, 28))
print('Sum ttb {}'.format(ttb.sum()))
ttb.to_series().head()

Sum ttb 0.00


1001     25646.21
1100     60958.94
1120       468.84
1200    -70576.88
1205         0.00
dtype: object

In [8]:
ttb_converter = TransactionalTrialBalance(start_year.chart_of_accounts)

In [9]:
ytd = start_year + ttb_converter.convert_trial_balance(ttb)

# Create account for the month
Having created YTD and MTD then upload into the database under temporary notation
Then load a report with the correct data and run the report.

In [10]:
ttb_converter = TransactionalTrialBalance(start_year.chart_of_accounts)
with LoadDatabase('historic_trial_balances.db') as ld:
    ttb = sage_2015.transactional_trial_balance(dt.date(2015, 2, 1), dt.date(2015, 2, 28))
    ytd = start_year + ttb_converter.convert_trial_balance(ttb)
    ld.load_tb_to_database(ytd, 'AYTD-FEB-15', overwrite=True)
    # TODO the signs of this are wrong it should be negative for 2107 so that in the BS
    # it comes out as positive

### Report on the new data

In [11]:
new_rep = PeriodReport(dt.datetime(2015, 2, 1), "historic_trial_balances.db", 
                       period_ytd_prefix="AYTD")

In [12]:
emr.create_one_report(new_rep)
new_rep.prior_mtd.to_series().tail()

8430    678.98
8433      0.00
8435    311.91
8440    860.04
9999      0.00
dtype: object

## Create accounts for the year
Turn into a function and Iterate for the whole year for 2015.

In [13]:
def get_period_open(dbname, coa, period_string):
    # Get data from historic trial balance database and store as TrialBalances
    with journal_from_db(dbname, coa, journal_entry_class=TrialBalance) as js:
        result = js.get_entry(period_string)  # YTD-FEB-15
    result = result.close_period()
    return result

In [14]:
def accounts_for_one_month(dbname, coa, year_open, month_open, start_of_year, start_of_month, end_of_month,
                          period):
    # Get opening trial balances for each perio
    open_year= get_period_open(dbname, coa, year_open)
    open_month= get_period_open(dbname, coa, month_open)
    ttb_converter = TransactionalTrialBalance(open_year.chart_of_accounts)
    with LoadDatabase(dbname) as ld:
        ytd_ttb = sage_2015.transactional_trial_balance(start_of_year, end_of_month)
        mtd_ttb = sage_2015.transactional_trial_balance(start_of_month, end_of_month)
        ytd = open_year + ttb_converter.convert_trial_balance(ytd_ttb)
        ld.load_tb_to_database(ytd, 'AYTD-{}'.format(period), overwrite=True)
        mtd = open_month + ttb_converter.convert_trial_balance(mtd_ttb)
        ld.load_tb_to_database(ytd, 'AMTD-{}'.format(period), overwrite=True)
    # TODO the signs of this are wrong it should be negative for 2107 so that in the BS
    # it comes out as positive

# Fiscal year constants
This would be better to calculate but quicker for two instances to just create the data table.  There are also exceptions.  Eg when do we switch from usingfinal versions of accounts to A.  This perhaps could be done programmatically.

In [15]:
data_2015 = (('YTD-JAN-15', 'YTD-JAN-15', dt.date(2015, 2, 1), dt.date(2015, 2, 1), dt.date(2015, 2, 28), 'FEB-15', ''),
       ('YTD-JAN-15', 'AYTD-FEB-15', dt.date(2015, 2, 1), dt.date(2015, 3, 1), dt.date(2015, 3, 31), 'MAR-15', ''),
       ('YTD-JAN-15', 'AYTD-MAR-15', dt.date(2015, 2, 1), dt.date(2015, 4, 1), dt.date(2015, 4, 30), 'APR-15', ''),
       ('YTD-JAN-15', 'AYTD-APR-15', dt.date(2015, 2, 1), dt.date(2015, 5, 1), dt.date(2015, 5, 31), 'MAY-15', ''),
       ('YTD-JAN-15', 'AYTD-MAY-15', dt.date(2015, 2, 1), dt.date(2015, 6, 1), dt.date(2015, 6, 30), 'JUN-15', ''),
       ('YTD-JAN-15', 'AYTD-JUN-15', dt.date(2015, 2, 1), dt.date(2015, 7, 1), dt.date(2015, 7, 31), 'JUL-15', ''),
       ('YTD-JAN-15', 'AYTD-JUL-15', dt.date(2015, 2, 1), dt.date(2015, 8, 1), dt.date(2015, 8, 31), 'AUG-15', ''),
       ('YTD-JAN-15', 'AYTD-AUG-15', dt.date(2015, 2, 1), dt.date(2015, 9, 1), dt.date(2015, 9, 30), 'SEP-15', ''),
       ('YTD-JAN-15', 'AYTD-SEP-15', dt.date(2015, 2, 1), dt.date(2015, 10, 1), dt.date(2015, 10, 31), 'OCT-15', ''),
       ('YTD-JAN-15', 'AYTD-OCT-15', dt.date(2015, 2, 1), dt.date(2015, 11, 1), dt.date(2015, 11, 30), 'NOV-15', ''),
       ('YTD-JAN-15', 'AYTD-NOV-15', dt.date(2015, 2, 1), dt.date(2015, 12, 1), dt.date(2015, 12, 31), 'DEC-15', ''),
       ('YTD-JAN-15', 'AYTD-DEC-15', dt.date(2015, 2, 1), dt.date(2016, 1, 1), dt.date(2016, 1, 31), 'JAN-16', '')
       )

In [16]:
dbname='historic_trial_balances.db'
with chart_of_accounts_from_db(dbname) as coa_s:
    coa = coa_s.get_chart_of_account('SLF-MA')
for year_open, month_open, start_of_year, start_of_month, end_of_month, period, prefix in data_2015:
    accounts_for_one_month(dbname, coa, year_open, month_open, start_of_year, start_of_month, end_of_month, period)

P&L = -45685.49
Prev retain = -509969.50, changing to -555654.99
New retained profit = -555654.99
Sum before clearing old balances = -45685.49
Sum after clearing old balances = 0.00
P&L = -45685.49
Prev retain = -509969.50, changing to -555654.99
New retained profit = -555654.99
Sum before clearing old balances = -45685.49
Sum after clearing old balances = 0.00
P&L = -45685.49
Prev retain = -509969.50, changing to -555654.99
New retained profit = -555654.99
Sum before clearing old balances = -45685.49
Sum after clearing old balances = 0.00
P&L = -12329.19
Prev retain = -555654.99, changing to -567984.18
New retained profit = -567984.18
Sum before clearing old balances = -12329.19
Sum after clearing old balances = 0.00
P&L = -45685.49
Prev retain = -509969.50, changing to -555654.99
New retained profit = -555654.99
Sum before clearing old balances = -45685.49
Sum after clearing old balances = 0.00
P&L = 2689.15
Prev retain = -555654.99, changing to -552965.84
New retained profit = -5529

## Print all reports for 2015

In [17]:
try:
    emr.open()
    for year_open, month_open, start_of_year, start_of_month, end_of_month, period, prior_prefix in data_2015:
        new_rep = PeriodReport(end_of_month, dbname,
                               period_ytd_prefix="AYTD",
                               period_mtd_prefix="AMTD",
                               prior_ytd_prefix="{}YTD".format(prior_prefix),
                               prior_mtd_prefix="{}MTD".format(prior_prefix)
                              )
        emr.add_sheet_pnl(new_rep)
        emr.add_sheet_bs(new_rep)
finally:
    emr.close()

## Using Jan 16 accounts generate accounts for 2016-2017
This can be redone once we have finalised the Jan 16 accounts

In [18]:
data_2016 = (('AYTD-JAN-16', 'AYTD-JAN-16', dt.date(2016, 2, 1), dt.date(2016, 2, 1), dt.date(2016, 2, 29), 'FEB-16', ''),
       ('AYTD-JAN-16', 'AYTD-FEB-16', dt.date(2016, 2, 1), dt.date(2016, 3, 1), dt.date(2016, 3, 31), 'MAR-16', ''),
       ('AYTD-JAN-16', 'AYTD-MAR-16', dt.date(2016, 2, 1), dt.date(2016, 4, 1), dt.date(2016, 4, 30), 'APR-16', ''),
       ('AYTD-JAN-16', 'AYTD-APR-16', dt.date(2016, 2, 1), dt.date(2016, 5, 1), dt.date(2016, 5, 31), 'MAY-16', ''),
       ('AYTD-JAN-16', 'AYTD-MAY-16', dt.date(2016, 2, 1), dt.date(2016, 6, 1), dt.date(2016, 6, 30), 'JUN-16', ''),
       ('AYTD-JAN-16', 'AYTD-JUN-16', dt.date(2016, 2, 1), dt.date(2016, 7, 1), dt.date(2016, 7, 31), 'JUL-16', ''),
       ('AYTD-JAN-16', 'AYTD-JUL-16', dt.date(2016, 2, 1), dt.date(2016, 8, 1), dt.date(2016, 8, 31), 'AUG-16', ''),
       ('AYTD-JAN-16', 'AYTD-AUG-16', dt.date(2016, 2, 1), dt.date(2016, 9, 1), dt.date(2016, 9, 30), 'SEP-16', ''),
       ('AYTD-JAN-16', 'AYTD-SEP-16', dt.date(2016, 2, 1), dt.date(2016, 10, 1), dt.date(2016, 10, 31), 'OCT-16', ''),
       ('AYTD-JAN-16', 'AYTD-OCT-16', dt.date(2016, 2, 1), dt.date(2016, 11, 1), dt.date(2016, 11, 30), 'NOV-16', ''),
       ('AYTD-JAN-16', 'AYTD-NOV-16', dt.date(2016, 2, 1), dt.date(2016, 12, 1), dt.date(2016, 12, 31), 'DEC-16', ''),
       ('AYTD-JAN-16', 'AYTD-DEC-16', dt.date(2016, 2, 1), dt.date(2017, 1, 1), dt.date(2017, 1, 31), 'JAN-17', '')
       )

In [19]:
with chart_of_accounts_from_db(dbname) as coa_s:
    coa = coa_s.get_chart_of_account('SLF-MA')
for year_open, month_open, start_of_year, start_of_month, end_of_month, period, prefix in data_2016:
    accounts_for_one_month(dbname, coa, year_open, month_open, start_of_year, start_of_month, end_of_month, period)

P&L = 9085.73
Prev retain = -555654.99, changing to -546569.26
New retained profit = -546569.26
Sum before clearing old balances = 9085.73
Sum after clearing old balances = 0.00
P&L = 9085.73
Prev retain = -555654.99, changing to -546569.26
New retained profit = -546569.26
Sum before clearing old balances = 9085.73
Sum after clearing old balances = 0.00
P&L = 9085.73
Prev retain = -555654.99, changing to -546569.26
New retained profit = -546569.26
Sum before clearing old balances = 9085.73
Sum after clearing old balances = 0.00
P&L = 20078.30
Prev retain = -546569.26, changing to -526490.96
New retained profit = -526490.96
Sum before clearing old balances = 20078.30
Sum after clearing old balances = 0.00
P&L = 9085.73
Prev retain = -555654.99, changing to -546569.26
New retained profit = -546569.26
Sum before clearing old balances = 9085.73
Sum after clearing old balances = 0.00
P&L = -46207.32
Prev retain = -546569.26, changing to -592776.58
New retained profit = -592776.58
Sum before

In [20]:
try:
    emr.open()
    for year_open, month_open, start_of_year, start_of_month, end_of_month, period, prior_prefix in data_2016:
        new_rep = PeriodReport(end_of_month, dbname,
                               period_ytd_prefix="AYTD",
                               period_mtd_prefix="AMTD",
                               prior_ytd_prefix="{}YTD".format(prior_prefix),
                               prior_mtd_prefix="{}MTD".format(prior_prefix)
                              )
        emr.add_sheet_pnl(new_rep)
        emr.add_sheet_bs(new_rep)
finally:
    emr.close()

LucaError: Getting Journal Entries from db historic_trial_balances.db for period MTD-MAY-15 but no data

# Playpit


In [ ]:
new_rep.period_ytd['TB']